In [18]:
import os
import numpy as np
import pandas as pd
import pickle
import cv2
from skimage.measure import label
from skimage.morphology import remove_small_objects

In [19]:
#inputs
pthDL = r'\\10.99.68.52\Kiemendata\Valentina Matos\tissues for methods paper\human liver\CODA_python_08_30_2024'
quantpath = r'\\10.99.68.52\Kiemendata\Valentina Matos\tissues for methods paper\human liver\10x\classification_CODA_python_08_30_2024'
tissue = [4, 2] #array with the annotation label that we want to quantify

In [20]:
#Load model data
with open(os.path.join(pthDL, 'net.pkl'), 'rb') as f:
    data = pickle.load(f)
    classNames = data['classNames']

In [21]:
# Define the path to the Excel file and the sheet name
excel_file = os.path.join(quantpath, 'image_quantifications.xlsx')
sheetName = 'Object analysis'

# Check if the Excel file exists
mode = 'w' if not os.path.exists(excel_file) else 'a'

# Write initial information to the Excel file
with pd.ExcelWriter(excel_file, engine='openpyxl', mode=mode) as writer:
    pd.DataFrame([['Model:', pthDL]]).to_excel(writer, sheet_name=sheetName, startrow=0, header=False, index=False)
    pd.DataFrame([['Image location:', quantpath]]).to_excel(writer, sheet_name=sheetName, startrow=1, header=False, index=False)
    pd.DataFrame([["Image", "Object ID", "Object Size (pixels)"]]).to_excel(writer, sheet_name=sheetName, startrow=2, header=False, index=False)

In [22]:
# Start object analysis
files = [os.path.join(quantpath, f) for f in os.listdir(quantpath) if f.endswith('.tif')]
print('_______Starting object analysis________')

all_props = []

for im_path in files:
    print(f'Processing image {os.path.basename(im_path)}')
    # Load image
    img = cv2.imread(im_path, cv2.IMREAD_GRAYSCALE)

    # Create a mask with the chosen annotation class to analyze
    for annotation_class in tissue:
        label_mask = img == annotation_class
        # Label objects
        labeled = label(label_mask, connectivity=1)
        labeled = remove_small_objects(labeled, min_size=500, connectivity=1)
        
        # Get object sizes
        for i in np.unique(labeled):
            if i != 0:
                all_props.append([os.path.basename(im_path), f"{classNames[label]} {i}", np.sum(labeled == i)])

# Convert to DataFrame
props_df = pd.DataFrame(all_props, columns=["Image", "Object ID", "Object Size (pixels)"])

# Write to Excel
with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a') as writer:
    props_df.to_excel(writer, sheet_name=sheetName, startrow=writer.sheets[sheetName].max_row, header=False, index=False)

print('_______Object analysis completed________')

_______Starting object analysis________
Processing image Liv-17_0001.tif


TypeError: list indices must be integers or slices, not function